In [31]:
import altair as alt
import pandas as pd
import geopandas as gpd 
alt.data_transformers.enable('json') 

pass

In [32]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
3273879,2,NADA,2013,75,9
3478518,2,ROSELINE,1963,70,3
3399880,2,PAULINE,1981,52,5
1726546,1,ZAKARIA,1995,59,6
3128971,2,MARILYNE,1963,21,3


In [33]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
90,90,Territoire de Belfort,"POLYGON ((6.82354 47.81305, 6.84618 47.82295, ..."
5,06,Alpes-Maritimes,"POLYGON ((6.88743 44.36105, 6.92257 44.35073, ..."
34,34,Hérault,"POLYGON ((3.35836 43.91383, 3.42445 43.91160, ..."
65,65,Hautes-Pyrénées,"MULTIPOLYGON (((-0.10308 43.24282, -0.12194 43..."
55,55,Meuse,"POLYGON ((4.95099 49.23687, 4.96436 49.24745, ..."


In [34]:
just_names = names
names = depts.merge(names, how='right', left_on='code', right_on='dpt')
names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
3111930,44,Loire-Atlantique,"POLYGON ((-2.45849 47.44812, -2.45343 47.46207...",2,MARYLINE,1956,44,25
1087984,55,Meuse,"POLYGON ((4.95099 49.23687, 4.96436 49.24745, ...",1,MARIO,1957,55,5
219234,72,Sarthe,"POLYGON ((-0.05453 48.38200, -0.04463 48.37976...",1,BERNARD,1958,72,57
2386312,92,Hauts-de-Seine,"POLYGON ((2.29097 48.95097, 2.32697 48.94536, ...",2,FARAH,1993,92,13
475161,59,Nord,"MULTIPOLYGON (((3.04040 50.15971, 3.06301 50.1...",1,ETIENNE,2013,59,9


### VIZ1

In [46]:
temp = names.groupby(['preusuel'], as_index=False).sum()[["preusuel","nombre"]]
temp = temp.sort_values(by=['nombre'])
most_popular = temp.tail(10)
most_unpopular = temp.head(10)

In [58]:
temp = names.groupby(['annais', 'preusuel','sexe'], as_index=False).sum()
evol_popular = temp[temp['preusuel'].isin(most_popular.preusuel.tolist())]
evol_unpopular = temp[temp['preusuel'].isin(most_unpopular.preusuel.tolist())]

In [100]:
d1 = alt.Chart(most_popular).mark_bar().encode(
    x='preusuel:O',
    y="nombre:Q",
)
d2 = alt.Chart(most_unpopular).mark_bar().encode(
    x='preusuel:O',
    y="nombre:Q",
)
d3 = alt.Chart(evol_popular).mark_line().encode(
    x="annais:O",
    y="nombre:Q",
    color='preusuel:O',
    strokeDash='preusuel:O',
).properties(width=2000)
d4 = alt.Chart(evol_unpopular).mark_line().encode(
    x="annais:O",
    y="nombre:Q",
    color='preusuel:O',
    strokeDash='preusuel:O',
).properties(width=800)

In [101]:
alt.vconcat(alt.hconcat(d1,d3), alt.hconcat(d2,d4))

alt.VConcatChart(...)

### VIZ2

In [102]:
temp = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
temp = depts.merge(temp, how='right', left_on='code', right_on='dpt') 
temp = temp[temp.preusuel.isin(most_popular.preusuel.tolist())]

In [121]:
selection = alt.selection_single(empty='all', fields=['preusuel'])
d1 = alt.Chart(most_popular).mark_bar().encode(
    x='preusuel:O',
    y="nombre:Q",
    color=alt.condition(selection,alt.value('green'),alt.value('lightgrey'))
).add_selection(selection)

In [122]:
map = alt.Chart(temp).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).transform_filter(selection
).properties(width=800, height=600,title='Heatmap of name popularity in France')

In [123]:
d1 & map

alt.VConcatChart(...)